In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Import packages.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

### Import the data


In [ ]:
mobility = pd.read_csv('/content/drive/My Drive/or_data/population.csv',index_col=0)

target_30_cities = pd.read_csv('/content/drive/My Drive/or_data/sample_submittion_round1 (1).csv',index_col=0)

In [ ]:
target_30_cities_list = list(target_30_cities.columns)

In [ ]:
target_30_cities.columns

Index(['福岡県福岡市中央区', '東京都千代田区', '島根県松江市', '埼玉県秩父郡長瀞町', '高知県吾川郡仁淀川町', '群馬県安中市',
       '青森県平川市', '宮城県栗原市', '北海道砂川市', '山梨県南都留郡道志村', '福島県南会津郡檜枝岐村',
       '長野県北佐久郡軽井沢町', '大阪府大阪市北区', '愛知県常滑市', '広島県三原市', '岐阜県大野郡白川村', '群馬県吾妻郡草津町',
       '北海道空知郡南富良野町', '千葉県千葉市美浜区', '千葉県浦安市', '新潟県南魚沼郡湯沢町', '茨城県鹿嶋市', '奈良県天理市',
       '秋田県仙北郡美郷町', '福井県勝山市', '佐賀県鳥栖市', '沖縄県那覇市', '滋賀県犬上郡甲良町', '東京都西多摩郡奥多摩町',
       '長野県東筑摩郡筑北村'],
      dtype='object')

In [ ]:
target_mobility = mobility[['福岡県福岡市中央区', '東京都千代田区', '島根県松江市', '埼玉県秩父郡長瀞町', '高知県吾川郡仁淀川町', '群馬県安中市',
       '青森県平川市', '宮城県栗原市', '北海道砂川市', '山梨県南都留郡道志村', '福島県南会津郡檜枝岐村',
       '長野県北佐久郡軽井沢町', '大阪府大阪市北区', '愛知県常滑市', '広島県三原市', '岐阜県大野郡白川村', '群馬県吾妻郡草津町',
       '北海道空知郡南富良野町', '千葉県千葉市美浜区', '千葉県浦安市', '新潟県南魚沼郡湯沢町', '茨城県鹿嶋市', '奈良県天理市',
       '秋田県仙北郡美郷町', '福井県勝山市', '佐賀県鳥栖市', '沖縄県那覇市', '滋賀県犬上郡甲良町', '東京都西多摩郡奥多摩町',
       '長野県東筑摩郡筑北村']]

In [ ]:
target_mobility

,福岡県福岡市中央区,東京都千代田区,島根県松江市,埼玉県秩父郡長瀞町,高知県吾川郡仁淀川町,群馬県安中市,青森県平川市,宮城県栗原市,北海道砂川市,山梨県南都留郡道志村,福島県南会津郡檜枝岐村,長野県北佐久郡軽井沢町,大阪府大阪市北区,愛知県常滑市,広島県三原市,岐阜県大野郡白川村,群馬県吾妻郡草津町,北海道空知郡南富良野町,千葉県千葉市美浜区,千葉県浦安市,新潟県南魚沼郡湯沢町,茨城県鹿嶋市,奈良県天理市,秋田県仙北郡美郷町,福井県勝山市,佐賀県鳥栖市,沖縄県那覇市,滋賀県犬上郡甲良町,東京都西多摩郡奥多摩町,長野県東筑摩郡筑北村
2019/01/01,243200.0,293700.0,210500.0,10100.0,5720.0,70700.0,38300.0,72300.0,22800.0,1770.0,590.0,28300.0,244000.0,71100.0,101500.0,2350.0,8500.0,2750.0,199800.0,206700.0,14800.0,79200.0,80900.0,24900.0,24000.0,95900.0,368400.0,11100.0,6300.0,5600.0
2019/01/02,267200.0,383700.0,211200.0,10700.0,5590.0,72000.0,39300.0,73400.0,24400.0,1970.0,660.0,28200.0,301000.0,74700.0,103800.0,2530.0,8600.0,2820.0,211600.0,210700.0,16200.0,78800.0,83500.0,24100.0,24900.0,103600.0,371300.0,12000.0,6400.0,5900.0
2019/01/03,269400.0,377700.0,212300.0,10600.0,5480.0,70800.0,38500.0,72300.0,24400.0,1990.0,540.0,27200.0,297000.0,72600.0,102600.0,2500.0,8400.0,2910.0,207300.0,208400.0,16600.0,77300.0,82300.0,24300.0,24700.0,104300.0,369100.0,10600.0,6300.0,5700.0
2019/01/04,304000.0,659700.0,214300.0,9900.0,5630.0,70400.0,38000.0,72500.0,24300.0,2000.0,500.0,26000.0,391000.0,73700.0,102400.0,2490.0,8500.0,2800.0,223200.0,216600.0,15500.0,78400.0,83800.0,23900.0,25000.0,103400.0,380400.0,11700.0,6200.0,5800.0
2019/01/05,288000.0,427700.0,211800.0,10100.0,5720.0,70800.0,39200.0,71900.0,24100.0,1920.0,480.0,25200.0,336000.0,72300.0,100400.0,2190.0,8000.0,2860.0,213100.0,212000.0,14600.0,77600.0,84300.0,23200.0,24500.0,101000.0,373600.0,10800.0,6300.0,5500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020/08/21,315000.0,761400.0,215400.0,10000.0,5830.0,70000.0,39000.0,74000.0,23300.0,2200.0,680.0,27500.0,438000.0,72100.0,103300.0,1850.0,7300.0,2990.0,230300.0,223500.0,10200.0,78500.0,82800.0,23900.0,25400.0,101600.0,375500.0,11600.0,6800.0,5170.0
2020/08/22,273700.0,343400.0,211200.0,10100.0,5630.0,68400.0,38300.0,72600.0,23300.0,2700.0,750.0,27700.0,300000.0,70000.0,100900.0,1900.0,7500.0,3000.0,208900.0,210400.0,10100.0,75700.0,78800.0,23900.0,25000.0,97000.0,366300.0,10600.0,7100.0,5090.0
2020/08/23,256500.0,260400.0,208200.0,9600.0,5690.0,66600.0,37400.0,71300.0,23000.0,2250.0,690.0,26800.0,261000.0,69600.0,99500.0,1830.0,7300.0,2960.0,199100.0,201100.0,9700.0,74300.0,76000.0,23400.0,24800.0,92700.0,356900.0,9900.0,6300.0,5050.0
2020/08/24,314000.0,783500.0,215300.0,9700.0,5900.0,69100.0,38500.0,73800.0,22900.0,1990.0,700.0,26400.0,437000.0,71500.0,103200.0,1780.0,7100.0,2860.0,228100.0,221100.0,10100.0,78500.0,82300.0,23800.0,25200.0,101200.0,371300.0,11400.0,6400.0,5180.0


### Dataloading
#### Use a slide window of length 4.

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

### Define the BiLSTM model.

In [ ]:
# Create an advanced model to beat the baseline

# Since the mean is a good predictor, add it to the model as a feature
# Use LSTM to discover patterns of variation from the mean

from keras.models import Sequential, Model
from keras import layers
from keras.optimizers import RMSprop
from keras import backend as K

i = layers.Input(shape = (4, 1), name ="original_feature" )

i_mean =  layers.Lambda(lambda x: K.mean(x, axis=-2, keepdims=True), name="mean")(i) 
i_wo_mean = layers.Subtract(name="variation_from_mean")([i, i_mean])
x = layers.BatchNormalization(name="batch_norm")(i_wo_mean)
x = layers.Bidirectional(layers.LSTM(10, dropout=0.5, \
                      recurrent_dropout = 0.5, \
                      input_shape=(4, 1)), name="bilstm")(x)
x = layers.Dropout(0.5, name="drop_out")(x) 
i_mean = layers.Flatten(name="flatten_mean")(i_mean)
f = layers.Concatenate(axis = -1, name="final_feature")([i_mean, x])
preds = layers.Dense(1, activation="relu", name="pred")(f)
model = Model(inputs = i, outputs = preds)

model.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
original_feature (InputLayer)   [(None, 4, 1)]       0                                            
__________________________________________________________________________________________________
mean (Lambda)                   (None, 1, 1)         0           original_feature[0][0]           
__________________________________________________________________________________________________
variation_from_mean (Subtract)  (None, 4, 1)         0           original_feature[0][0]           
                                                                 mean[0][0]                       
__________________________________________________________________________________________________
batch_norm (BatchNormalization) (None, 4, 1)         4           variation_from_mean[0]

### Define a function to get the RMSE of the last 120 days' prediction and its population mobility prediction for the first half of September, 2020. 

In [ ]:
def RMSE_prediction_city(city_name, dataset):
    
    from numpy.random import seed
    seed(123)
    # Obtain the population mobility of the specified city.
    sc = MinMaxScaler()
    training_data = sc.fit_transform(target_mobility[city_name].values.reshape(-1,1))

    seq_length = 4
    x, y = sliding_windows(training_data, seq_length)
    # Create training and testing dataset by splitting the time series into 3 contiguous parts(training, validation,testing)
    # Keep the population mobility of the last 120 days until 2020-08-25 for testing and the rest for training and validation.
    
    train_size = int((len(y)-120) * 0.80)
    val_size = int(len(y)-120) - int((len(y)-120) * 0.80)
    test_size = 120

    train_X = x[:479]
    train_Y = y[:479]
    

    test_X = x[-120:]
    test_Y = y[-120:]

    model.compile(optimizer=RMSprop(), loss='mse')
    history = model.fit(train_X, train_Y,\
                    batch_size = 64, epochs = 100,\
                   )
    
    preds = model.predict(test_X)
    preds = sc.inverse_transform(preds)
    preds = preds.reshape(120,)
    actual_data = target_mobility[city_name][483:].values

    rmse = np.mean((preds - actual_data)**2)**.5 
    # from sklearn.metrics import mean_squared_error 
    # rmse_2 = mean_squared_error(actual_data, preds)**0.5

    new_days_predict = []
    training_data_copy = training_data

    for i in range(21):
      dataX_new = np.array(training_data_copy[-4:].reshape(1,4,1))
      new_predict = model.predict(dataX_new)

      
      training_data_copy = np.append(training_data_copy, new_predict)
      new_predict_real = sc.inverse_transform(new_predict)
      new_days_predict.append(new_predict_real[0][0])

    final_predcition = new_days_predict[6:]

    
    return ({ 
            'rmse_BiLSTM':rmse,
            'prediction_BiLSTM': final_predcition
           })
    

In [ ]:
f = RMSE_prediction_city('福岡県福岡市中央区',target_mobility)
f

Epoch 1/100
8/8 [==============================] - 0s 8ms/step - loss: 0.0285
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0289
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0268
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0266
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0280
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0292
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0281
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0272
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0275
Epoch 10/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0280
Epoch 11/100
8/8 [==============================] - 0s 9ms/step - loss: 0.0283
Epoch 12/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0277
Epoch 13/100
8/8 [==============================] - 0s 7ms/st

{'prediction_BiLSTM': [306522.34,
  309263.25,
  308528.34,
  307418.97,
  304830.62,
  301994.7,
  300331.8,
  300320.88,
  301191.53,
  302310.84,
  303452.7,
  304313.2,
  304663.6,
  304446.97,
  303759.4],
 'rmse_BiLSTM': 23865.607367407065}

- Get a list of dictionaries and each dictionary contains RMSE for the prediction of corresponding city and its prediction on the population mobility of first half of September. The list contains all 30 cities.

In [ ]:
import time
start_time = time.time()
RMSE_prediction_30_cities = [RMSE_prediction_city(x,target_mobility) for x in target_30_cities_list]

RMSE_prediction_30_cities
print("--- %s seconds ---" % (time.time() - start_time))

Streaming output truncated to the last 5000 lines.
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0043
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0045
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0043
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0045
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0042
Epoch 7/100
8/8 [==============================] - 0s 11ms/step - loss: 0.0042
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0042
Epoch 9/100
8/8 [==============================] - 0s 7ms/step - loss: 0.0044
Epoch 10/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0043
Epoch 11/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0041
Epoch 12/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0044
Epoch 13/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0042
Epoch 14

### Transform the list of dictionaries to a dataframe and add a new column of city_name as the first column of the dataframe. 

In [ ]:
df_rmse_prediction_BiLSTM = pd.DataFrame(RMSE_prediction_30_cities)




idx = 0
new_col = target_30_cities_list  
df_rmse_prediction_BiLSTM.insert(loc=idx, column='city name', value=new_col)
df_rmse_prediction_BiLSTM

,city name,rmse_BiLSTM,prediction_BiLSTM
0,福岡県福岡市中央区,22843.502496,"[310484.66, 309832.03, 308402.56, 303503.47, 2..."
1,東京都千代田区,209644.464203,"[762922.8, 838871.6, 917709.1, 929603.3, 82100..."
2,島根県松江市,2372.061163,"[214978.31, 214618.52, 214170.23, 213790.86, 2..."
3,埼玉県秩父郡長瀞町,346.805075,"[10153.641, 10170.155, 10182.802, 10192.786, 1..."
4,高知県吾川郡仁淀川町,111.992470,"[5805.083, 5803.5684, 5801.4883, 5800.0854, 57..."
5,群馬県安中市,1247.541958,"[70094.25, 70171.67, 70241.68, 70305.875, 7036..."
6,青森県平川市,562.354469,"[38755.758, 38797.53, 38832.184, 38861.55, 388..."
7,宮城県栗原市,672.471412,"[73241.766, 72916.31, 72639.945, 72751.336, 72..."
8,北海道砂川市,430.195572,"[23682.621, 23749.861, 23813.25, 23873.438, 23..."
9,山梨県南都留郡道志村,312.617082,"[2238.0964, 2258.0217, 2272.4438, 2284.227, 22..."


### Save the results.

In [ ]:
df_rmse_prediction_BiLSTM.to_csv('/content/drive/My Drive/or_data/df_rmse_prediction_BiLSTM.csv',index=False,encoding='utf_8_sig')

In [ ]:
len(df_rmse_prediction_BiLSTM['prediction_BiLSTM'][0])

15

In [ ]:
df_rmse_prediction_BiLSTM['prediction_BiLSTM'][0]

[310484.66,
 309832.03,
 308402.56,
 303503.47,
 295877.25,
 291411.03,
 295544.72,
 299924.25,
 301024.56,
 301955.53,
 300604.1,
 297312.47,
 293588.94,
 291840.03,
 292984.8]